In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import hashlib
import random
import os
import pickle
import joblib
os.chdir('Resources/')

In [2]:
with open('3_Public_Key.pkl', 'rb') as f_public:
    public_key_data = pickle.load(f_public)

q = public_key_data['q']
g = public_key_data['g']
public_keys = public_key_data['public_keys']

In [3]:
model = joblib.load('9_RF_Isolation_Model.joblib')
scaler = joblib.load('6_scaler.joblib')

In [4]:
def power(a, b, c):
    x = 1
    y = a
    while b > 0:
        if b % 2 != 0:
            x = (x * y) % c
        y = (y * y) % c
        b = int(b / 2)
    return x % c

In [5]:
def encrypt(msg, q, h, g):
    en_msg = []
    k = gen_key(q)  # Private key for sender
    s = power(h, k, q)
    p = power(g, k, q)
    
    for i in range(0, len(msg)):
        en_msg.append(s * msg[i])  # Encrypt each element
    
    return en_msg, p

In [8]:
def save_button_clicked(button):
    X_new_s = {
        'Age': int(age_widget.value),
        'Sex': sex_widget.value,
        'ChestPainType': chest_pain_type_widget.value,
        'RestingBloodPressure': int(resting_blood_pressure_widget.value),
        'Cholesterol': int(cholesterol_widget.value),
        'FastingBloodSugar': fasting_blood_sugar_widget.value,
        'RestingECG': resting_ecg_widget.value,
        'MaximumHeartRate': int(max_heart_rate_widget.value),
        'ExerciseAngina': exercise_angina_widget.value,
        'Oldpeak': float(oldpeak_widget.value),
        'ST_Slope': st_slope_widget.value
    }

    encrypted_row = {}
    for column, value in X_new_s.items():
        q, h, g = column_keys[column]
        if isinstance(value, str):
            numeric_value = sum(ord(char) for char in value)
            encrypted_value, _ = encrypt(numeric_value, q, h, g, fixed_key)
        else:
            encrypted_value, _ = encrypt(float(value), q, h, g, fixed_key)
        encrypted_row[column] = encrypted_value

    encrypted_df_s = pd.DataFrame([encrypted_row])

    X_new_scaled_s = scaler.transform(encrypted_df_s)

    pred_new_s = model.predict(X_new_scaled_s)

    output_widget.clear_output()
    with output_widget:
        if pred_new_s[0] == 0:
            print("Heart Disease: No")
        else:
            print("Heart Disease: Yes")

In [9]:
age_widget = widgets.Text(description="Age:")
sex_widget = widgets.Dropdown(options=['Female', 'Male'], description="Sex:")
chest_pain_type_widget = widgets.Dropdown(options=['Typical Angina', 'Atypical Angina', 'Non-anginal Pain', 'Asymptomatic'], description="ChestPainType:")
resting_blood_pressure_widget = widgets.Text(description="RestingBloodPressure:")
cholesterol_widget = widgets.Text(description="Cholesterol:")
fasting_blood_sugar_widget = widgets.Dropdown(options=['No', 'Yes'], description="FastingBloodSugar:")
resting_ecg_widget = widgets.Dropdown(options=['Normal', 'Abnormal ST-T Wave', 'Left Ventricular Hypertrophy'], description="RestingECG:")
max_heart_rate_widget = widgets.Text(description="MaximumHeartRate:")
exercise_angina_widget = widgets.Dropdown(options=['No', 'Yes'], description="ExerciseAngina:")
oldpeak_widget = widgets.Text(description="Oldpeak:")
st_slope_widget = widgets.Dropdown(options=['Upsloping', 'Flat', 'Downsloping'], description="ST_Slope:")

save_button = widgets.Button(description="Save Data")
save_button.on_click(save_button_clicked)

output_widget = widgets.Output()

display(age_widget, sex_widget, chest_pain_type_widget, resting_blood_pressure_widget, cholesterol_widget, fasting_blood_sugar_widget, resting_ecg_widget, max_heart_rate_widget, exercise_angina_widget, oldpeak_widget, st_slope_widget, save_button, output_widget)

Text(value='', description='Age:')

Dropdown(description='Sex:', options=('Female', 'Male'), value='Female')

Dropdown(description='ChestPainType:', options=('Typical Angina', 'Atypical Angina', 'Non-anginal Pain', 'Asym…

Text(value='', description='RestingBloodPressure:')

Text(value='', description='Cholesterol:')

...

Text(value='', description='Oldpeak:')

Dropdown(description='ST_Slope:', options=('Upsloping', 'Flat', 'Downsloping'), value='Upsloping')

Button(description='Save Data', style=ButtonStyle())

Output()

NameError: name 'column_keys' is not defined

In [1]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
import os
os.chdir('Resources/')

# Load the secret data
df = pd.read_csv('8_SP_Encrypted_Data.csv')
X = df.drop(['HeartDisease'], axis='columns')
Y = df[['HeartDisease']]

# Initialize lists to store the true and predicted labels
y_true = []
y_pred = []

# Set up the stratified k-fold cross-validation (10 folds)
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=530)

# Iterate through each fold
for train_index, test_index in kf.split(df, Y):
    # Split the data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

    # Train the model
    model.fit(X_train, Y_train)
    
    # Predict on the test set
    y_pred_fold = model.predict(X_test)

    # Append the true labels and predicted labels for this fold
    y_true.extend(Y_test)
    y_pred.extend(y_pred_fold)

# Calculate the F1 score for all folds
f1 = f1_score(y_true, y_pred) * 100
print(f"Average F1 Score (10-fold): {f1}")


NameError: name 'model' is not defined